We consider a circuit of the form

![image.png](attachment:image.png)

The dynamics are given by

$\dot p_i = \frac{\alpha_i}{J_i} \frac{d_i}{1 + d_1 + d_2} - \delta p_i$

Assuming that there is a constant $c$ such that $\alpha_i = c J_i$, the steady-state behavior is

$\frac{\delta}{c} p_i = \frac{d_i}{1 + d_1 + d_2}$

The relation between the $d_i$ and the $J_i$ is given by
$d_1 = J_1 F_1(u)$
$d_2 = J_2 F_2(p_1)$

We will assume that the $F_i$ are monotonically increasing.

We would like to state two requirements for our circuit:
- When the input $u$ exceeds a threshold $u^H$, the output $p_2$ should exceed a threshold $p_{2}^{H}$
- When the input $u$ is below a threshold $u^L$, the output $p_2$ should be below a threshold $p_{2}^{L}$.

### Upper threshold

- In addition to $p_2$ having a lower threshold $p_{2H}$, we will assume it does not exceed a maximum value $p_{2}^{max}$.
- We will assume that $p_1$ must exceed a lower threshold $p_{1H}$ and that it has a maximum value $p_{1}^{max}$.
- We will assume $u^{max}$ is the max value of $u$.

From the previous equations, we have two constraints:

$\frac{d_i}{1 + d_1 + d_2} \ge \frac{\delta}{c} p_{i}^{H}$

which becomes

$-d_i + \frac{\delta p_{i}^{H}}{c} d_1 + \frac{\delta p_{i}^{H}}{c} d_2 \le -\frac{\delta p_{i}^{H}}{c}$

Moreover, we have the constraints

$J_1 F_1(u^{max}) \ge d_1 \ge J_1 F_1(u^H)$

$J_2 F_2(p_1^{max}) \ge d_2 \ge J_2 F_2(p_1^H)$

We model the problem as follows:

In [2]:
import logging

FORMAT = "%(asctime)s:%(levelname)s:%(name)s:%(message)s"
logging.basicConfig(filename="../pacti.log", filemode="w", level=logging.DEBUG, format=FORMAT)

## constants
delta = 1
c = 1
# sigma 1
p1_H = 0.9
F_1_u_max = 1
F_1_u_H = 0.9
# sigma 2
p2_H = 0.91
F_2_p_1_max = 1
F_2_p_1_H = 0.8


from pacti.terms.polyhedra import PolyhedralContract

top_level_objective = PolyhedralContract.from_string(
    input_vars=[],
    output_vars=["d1", "d2"],
    assumptions=[],
    guarantees=[
    f"{-1 + (delta * p1_H / c)} d1 + {delta * p1_H / c} d2 <= -{delta * p1_H / c}",
    f"{delta * p2_H / c} d1  {-1 + (delta * p2_H / c)} d2 <= -{delta * p2_H / c}"
    ]
    )

sigma1 = PolyhedralContract.from_string(
    input_vars=["d2"],
    output_vars=["d1", "J1"],
    assumptions=[],
    guarantees=[
    f"d1 - {F_1_u_max} J1  <= 0",
    f"{F_1_u_H} J1 - d1 <= 0"
    ],
    )

sigma2 = PolyhedralContract.from_string(
    input_vars=["d1"],
    output_vars=["d2", "J2"],
    assumptions=[],
    guarantees=[
    f"d2 - {F_2_p_1_max} J2  <= 0",
    f"{F_2_p_1_H} J2 - d2 <= 0"
    ]
    )


system_spec = sigma1.compose(sigma2, vars_to_keep=["d1","d2"])
print(f"== Top-level spec\n{top_level_objective}")
print(f"== Composed spec\n{system_spec}")

system_H_threshold = top_level_objective.quotient(system_spec)
print(system_H_threshold)

== Top-level spec
InVars: []
OutVars:[d1, d2]
A: [
  
]
G: [
  -0.09999999999999998 d1 + 0.9 d2 <= -0.9
  0.91 d1 - 0.08999999999999997 d2 <= -0.91
]
== Composed spec
InVars: []
OutVars:[J1, J2, d1, d2]
A: [
  
]
G: [
  -J1 + d1 <= 0.0
  0.9 J1 - d1 <= 0.0
  -J2 + d2 <= 0.0
  0.8 J2 - d2 <= 0.0
]
inputvars0 []
inputvars2 [<Var J1>, <Var J2>]
inputvars3 [<Var J1>, <Var J2>]


IncompatibleArgsError: Could not eliminate variables 
['d1', 'd2']by refining the guarantees 
[
  -0.09999999999999998 d1 + 0.9 d2 <= -0.9
  0.91 d1 - 0.08999999999999997 d2 <= -0.91
]


The composition of these two subsystems $\Sigma_i$ gave us a 